<a href="https://colab.research.google.com/github/larajakl/Computational-Linguistics/blob/main/Project_Lara_JAKL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets
!pip install evaluate
!pip install accelerate --upgrade
!pip install optuna
!pip install optuna-integration[pytorch_lightning]

In [ ]:
from datasets import load_dataset, DatasetDict
from transformers import DataCollatorWithPadding

from transformers import AutoTokenizer

from transformers import set_seed
from collections import Counter


In [ ]:
dataset = load_dataset("mrjunos/depression-reddit-cleaned")

set_seed(24)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Check distribution of labels in full dataset:

full_label_distribution = Counter(dataset['train']['label'])
print("Full dataset label distribution:", full_label_distribution)

Full dataset label distribution: Counter({0: 3900, 1: 3831})


In [ ]:
# Truncation function and shuffling and splitting of dataset:
def truncate(example):
    return {
        'text': " ".join(example['text'].split()[:300]),
        'label': example['label']
    }

# Random examples for train, validation and test
# Limit the dataset to the first 200 entries, JUST FOR NOW (ADAPT THESE LINES LATER)
subset_dataset = dataset['train'].shuffle(seed=24).select(range(1250))
# Define the train/val/test split proportions:
train_ratio, val_ratio = 0.7, 0.15  # 70% train, 15% val, 15% test
# Shuffle the dataset once:
shuffled_dataset = subset_dataset.shuffle(seed=24)
# Compute the split indices:
total_size = len(shuffled_dataset)
train_end = int(train_ratio * total_size)
val_end = train_end + int(val_ratio * total_size)
# Create splits:
train = shuffled_dataset.select(range(train_end)).map(truncate)
val = shuffled_dataset.select(range(train_end, val_end)).map(truncate)
test = shuffled_dataset.select(range(val_end, total_size)).map(truncate)

# Print the sizes of the splits:
print(f"Train size: {len(train)}, Validation size: {len(val)}, Test size: {len(test)}")

dataset_dict = DatasetDict({
    "train": train,
    "val": val,
    "test": test
})

Train size: 875, Validation size: 187, Test size: 188


In [ ]:
# To check structure of dataset:
print(shuffled_dataset)
print(dataset_dict)

Dataset({
    features: ['text', 'label'],
    num_rows: 1250
})
DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 875
    })
    val: Dataset({
        features: ['text', 'label'],
        num_rows: 187
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 188
    })
})


In [ ]:
# Model 1: Tokenizer DistilBERT cased
tokenizer_distilbert = AutoTokenizer.from_pretrained("distilbert/distilbert-base-cased")

def tokenize_function_distilbert(examples):
    return tokenizer_distilbert(examples["text"], padding=True, truncation=True)

small_tokenized_dataset_distilbert = dataset_dict.map(tokenize_function_distilbert, batched=True, batch_size=16)
data_collator_distilbert = DataCollatorWithPadding(tokenizer=tokenizer_distilbert)

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

In [ ]:
print(small_tokenized_dataset_distilbert)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 875
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 187
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 188
    })
})


In [ ]:
# Model 2: Tokenizer RoBERTa base
tokenizer_roberta = AutoTokenizer.from_pretrained("roberta-base")

def tokenize_function_roberta(examples):
    return tokenizer_roberta(examples["text"], padding=True, truncation=True)

# Apply the tokenize function to the dataset
small_tokenized_dataset_roberta = dataset_dict.map(tokenize_function_roberta, batched=True, batch_size=16)

# Create a data collator with padding
data_collator_roberta = DataCollatorWithPadding(tokenizer=tokenizer_roberta)

Map:   0%|          | 0/187 [00:00<?, ? examples/s]

In [ ]:
# Mounting Google Drive to store the checkpoints in Google Drive instead of my runtime:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForSequenceClassification

In [ ]:
# Training the Distil BERT cased model:

set_seed(24)

model_distilbert = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2) # 2 labels: depression/no depression
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

arguments_distilbert = TrainingArguments(
    output_dir="distilbert",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    logging_steps=10,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_recall", # so that the model that performs best in the recall score is loaded at the end
    learning_rate=3e-5,
    weight_decay=0.1,
    overwrite_output_dir=True,
    load_best_model_at_end=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Debugging: Print shapes and values
    #print(f"Logits shape: {logits.shape}")
    #print(f"Labels: {labels}")
    #print(f"Predictions: {predictions}")

    # Scikit-learn for metric computation
    acc = accuracy_score(labels, predictions)
    prec = precision_score(labels, predictions, average="binary", zero_division=0)
    rec = recall_score(labels, predictions, average="binary", zero_division=0)
    f1 = f1_score(labels, predictions, average="binary", zero_division=0)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }


trainer_distilbert = Trainer(
    model=model_distilbert,
    args=arguments_distilbert,
    train_dataset=small_tokenized_dataset_distilbert['train'],
    eval_dataset=small_tokenized_dataset_distilbert['val'],
    processing_class=tokenizer_distilbert,
    data_collator=data_collator_distilbert,
    compute_metrics=compute_metrics
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer_distilbert.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.119400,0.129073,0.957219,0.978261,0.937500,0.957447
2,0.053900,0.121382,0.957219,0.968085,0.947917,0.957895
3,0.046400,0.135013,0.962567,0.968421,0.958333,0.963351
4,0.041200,0.155069,0.973262,1.000000,0.947917,0.973262
5,0.017400,0.167772,0.962567,0.978495,0.947917,0.962963


Logits shape: (187, 2)
Labels: [0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1
 1 1 1 1 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1
 0 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0
 0 1 0 1 0 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1 1 1 0 1 0 1 0 0 0 1 1 1
 1 1]
Predictions: [0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 0 1 1 0 1 1 1 0 1 1 1 0 1 0 0 1 0 1 1
 0 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 0 0 1 0 1 1 0 0 0 0 0 0 1 0 1 0 1 1 0 1 0
 0 1 0 1 1 1 0 1 1 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 1 0 1 0 1 0 0 0 0 1 1
 1 1]
Logits shape: (187, 2)
Labels: [0 0 1 1 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 1 1 1 0
 0 1 1 0 0 1 0 0 0 0 1 0 0 1 1 1 1 0 0 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 1
 1 1 1 1 0 0 

TrainOutput(global_step=275, training_loss=0.09883375390009447, metrics={'train_runtime': 179.701, 'train_samples_per_second': 24.346, 'train_steps_per_second': 1.53, 'total_flos': 391548016613724.0, 'train_loss': 0.09883375390009447, 'epoch': 5.0})

In [ ]:
# To see which model was actually the best one (which checkpoint):
print(trainer_distilbert.state.best_model_checkpoint)

distilbert/checkpoint-165


In [ ]:
# Training the RoBERTa base model:

set_seed(24)

model_roberta = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2) # 2 labels: depression/no depression
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

arguments_roberta = TrainingArguments(
    output_dir="roberta",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    logging_steps=10,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="eval_recall",
    learning_rate=3e-5,
    weight_decay=0.02,
    load_best_model_at_end=True,
    overwrite_output_dir=True,
    report_to='none',
    seed=224
)

def compute_metrics(eval_pred):
    """Called at the end of validation."""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    # Debugging: Print shapes and values
    #print(f"Logits shape: {logits.shape}")
    #print(f"Labels: {labels}")
    #print(f"Predictions: {predictions}")

    # Scikit-learn for metric computation
    acc = accuracy_score(labels, predictions)
    prec = precision_score(labels, predictions, average="binary", zero_division=0)
    rec = recall_score(labels, predictions, average="binary", zero_division=0)
    f1 = f1_score(labels, predictions, average="binary", zero_division=0)

    return {
        "accuracy": acc,
        "precision": prec,
        "recall": rec,
        "f1": f1
    }


trainer_roberta = Trainer(
    model=model_roberta,
    args=arguments_roberta,
    train_dataset=small_tokenized_dataset_roberta['train'],
    eval_dataset=small_tokenized_dataset_roberta['val'],
    processing_class=tokenizer_roberta,
    data_collator=data_collator_roberta,
    compute_metrics=compute_metrics
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# To check the label distribution:
print("Train label distribution:", Counter(train['label']))
print("Validation label distribution:", Counter(val['label']))
print("Test label distribution:", Counter(test['label']))

Train label distribution: Counter({0: 442, 1: 433})
Validation label distribution: Counter({1: 96, 0: 91})
Test label distribution: Counter({1: 98, 0: 90})


In [ ]:
trainer_roberta.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.142300,0.154840,0.962567,0.989011,0.937500,0.962567
2,0.082200,0.169443,0.962567,1.000000,0.927083,0.962162
3,0.084100,0.246036,0.957219,0.968085,0.947917,0.957895
4,0.040100,0.239469,0.957219,0.968085,0.947917,0.957895
5,0.036500,0.235300,0.957219,0.968085,0.947917,0.957895


TrainOutput(global_step=275, training_loss=0.10542573403567075, metrics={'train_runtime': 345.7583, 'train_samples_per_second': 12.653, 'train_steps_per_second': 0.795, 'total_flos': 734661566553360.0, 'train_loss': 0.10542573403567075, 'epoch': 5.0})

In [ ]:
# To see which model was actually the best one (which checkpoint):
print(trainer_roberta.state.best_model_checkpoint)

/content/drive/MyDrive/roberta/checkpoint-165


In [ ]:
print(small_tokenized_dataset_roberta)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 875
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 187
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 188
    })
})


In [ ]:
print(small_tokenized_dataset_distilbert)

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 875
    })
    val: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 187
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 188
    })
})


In [ ]:
'''
This code cell was used to fine-tune the weight decay hyperparameter, and does not need to be run again.
'''
# Fine-tuning weight decay for distilbert:
import tempfile

# Temporary directory for saving models during training
temp_dir = tempfile.mkdtemp()

model_distilbert = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2) # 2 labels: depression/no depression

weight_decay_values = [0.001, 0.005, 0.01, 0.02, 0.1]
for wd in weight_decay_values:
    model_distilbert = AutoModelForSequenceClassification.from_pretrained('distilbert/distilbert-base-cased', num_labels=2) # 2 labels: depression/no depression

    arguments_distilbert = TrainingArguments(
    output_dir=temp_dir,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    logging_steps=10,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    weight_decay=wd,
    load_best_model_at_end=False,
    report_to='none',
    seed=224,
    )

    trainer = Trainer(
    model=model_distilbert,
    args=arguments_distilbert,
    train_dataset=small_tokenized_dataset_distilbert['train'],
    eval_dataset=small_tokenized_dataset_distilbert['val'],
    processing_class=tokenizer_distilbert,
    data_collator=data_collator_distilbert,
    compute_metrics=compute_metrics
    )

    trainer.train()

In [ ]:
'''
This code cell was used to fine-tune the weight decay hyperparameter, and does not need to be run again.
'''
# Fine-tuning weight decay RoBERTa:

model_roberta = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2) # 2 labels: depression/no depression

weight_decay_values = [0.001, 0.005, 0.01, 0.02, 0.1]
for wd in weight_decay_values:
    model_roberta = AutoModelForSequenceClassification.from_pretrained('roberta-base', num_labels=2) # 2 labels: depression/no depression

    arguments_roberta = TrainingArguments(
    output_dir=temp_dir,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    logging_steps=10,
    num_train_epochs=5,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    weight_decay=wd,
    load_best_model_at_end=False,
    report_to='none',
    seed=224
    )

    trainer = Trainer(
    model=model_roberta,
    args=arguments_roberta,
    train_dataset=small_tokenized_dataset_roberta['train'],
    eval_dataset=small_tokenized_dataset_roberta['val'],
    processing_class=tokenizer_roberta,
    data_collator=data_collator_roberta,
    compute_metrics=compute_metrics
    )

    trainer.train()

In [ ]:
import os
import torch

In [ ]:
# Testing the final DistilBERT model saved in Drive with Trainer.evaluate() (less memory intensive):

fine_tuned_model_distilbert = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/distilbert/checkpoint-165")

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

# Create a new Trainer instance for evaluation
new_trainer = Trainer(
    model=fine_tuned_model_distilbert,
    eval_dataset=small_tokenized_dataset_distilbert["test"],
    compute_metrics=compute_metrics,
)

# Evaluate the model
test_results = new_trainer.evaluate()
print(test_results)

# Get predictions and labels
predictions, labels, _ = new_trainer.predict(small_tokenized_dataset_distilbert["test"])

# Get the predicted class (most probable class)
predicted_classes = torch.argmax(torch.tensor(predictions), dim=-1)

# Find wrongly classified samples
wrongly_classified = []
for i in range(len(predicted_classes)):
    if predicted_classes[i] != labels[i]:
        wrongly_classified.append({
            "text": small_tokenized_dataset_distilbert["test"][i]["text"],
            "true_label": labels[i],
            "predicted_label": predicted_classes[i]
        })

# Output the wrongly classified samples
for example in wrongly_classified:
    print(f"Text: {example['text']}")
    print(f"True Label: {example['true_label']}")
    print(f"Predicted Label: {example['predicted_label']}")
    print("-" * 100)

# Evaluate the model on the entire test set
test_results = new_trainer.evaluate()
print(test_results)

# This gives me results for best DistilBERT model from Drive.

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


{'eval_loss': 0.0935538038611412, 'eval_model_preparation_time': 0.0024, 'eval_accuracy': 0.973404255319149, 'eval_precision': 0.9894736842105263, 'eval_recall': 0.9591836734693877, 'eval_f1': 0.9740932642487047, 'eval_runtime': 4.7309, 'eval_samples_per_second': 39.739, 'eval_steps_per_second': 5.073}
Text: scared that the ceiling fan will drop and decapitate me can t turn on the air condioner either scared that it might explode
True Label: 1
Predicted Label: 0
----------------------------------------------------------------------------------------------------
Text: day have passed since i last posted nothing ha improved my friend just hang with me for my stuff my family see me a a liability and useless it s midnight again and i wish i wa dead
True Label: 1
Predicted Label: 0
----------------------------------------------------------------------------------------------------
Text: i feel so much worse after talking to a counseller i swear this time i might actually do it
True Label: 1

{'eval_loss': 0.0935538038611412, 'eval_model_preparation_time': 0.0024, 'eval_accuracy': 0.973404255319149, 'eval_precision': 0.9894736842105263, 'eval_recall': 0.9591836734693877, 'eval_f1': 0.9740932642487047, 'eval_runtime': 1.4782, 'eval_samples_per_second': 127.182, 'eval_steps_per_second': 16.236}


In [ ]:
# Testing the final RoBERTa model saved in Drive with Trainer.evaluate() (less memory intensive):

fine_tuned_model_roberta = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/roberta/checkpoint-165")

# Disable W&B logging
os.environ["WANDB_DISABLED"] = "true"

# Create a new Trainer instance for evaluation
new_trainer = Trainer(
    model=fine_tuned_model_roberta,
    eval_dataset=small_tokenized_dataset_roberta["test"],
    compute_metrics=compute_metrics,
)

# Evaluate the model
test_results = new_trainer.evaluate()
print(test_results)

# Get predictions and labels
predictions, labels, _ = new_trainer.predict(small_tokenized_dataset_roberta["test"])

# Get the predicted class (most probable class)
predicted_classes = torch.argmax(torch.tensor(predictions), dim=-1)

# Find wrongly classified samples
wrongly_classified = []
for i in range(len(predicted_classes)):
    if predicted_classes[i] != labels[i]:
        wrongly_classified.append({
            "text": small_tokenized_dataset_roberta["test"][i]["text"],
            "true_label": labels[i],
            "predicted_label": predicted_classes[i]
        })

# Output the wrongly classified samples
for example in wrongly_classified:
    print(f"Text: {example['text']}")
    print(f"True Label: {example['true_label']}")
    print(f"Predicted Label: {example['predicted_label']}")
    print("-" * 100)

# Evaluate the model on the entire test set
test_results = new_trainer.evaluate()
print(test_results)

# This gives me results for best RoBERTa model from Drive.

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


{'eval_loss': 0.1415235996246338, 'eval_model_preparation_time': 0.0027, 'eval_accuracy': 0.9787234042553191, 'eval_precision': 0.9895833333333334, 'eval_recall': 0.9693877551020408, 'eval_f1': 0.979381443298969, 'eval_runtime': 2.5885, 'eval_samples_per_second': 72.628, 'eval_steps_per_second': 9.272}
Text: scared that the ceiling fan will drop and decapitate me can t turn on the air condioner either scared that it might explode
True Label: 1
Predicted Label: 0
----------------------------------------------------------------------------------------------------
Text: day have passed since i last posted nothing ha improved my friend just hang with me for my stuff my family see me a a liability and useless it s midnight again and i wish i wa dead
True Label: 1
Predicted Label: 0
----------------------------------------------------------------------------------------------------
Text: need s help with this anxiety crap
True Label: 0
Predicted Label: 1
-------------------------------------

{'eval_loss': 0.1415235996246338, 'eval_model_preparation_time': 0.0027, 'eval_accuracy': 0.9787234042553191, 'eval_precision': 0.9895833333333334, 'eval_recall': 0.9693877551020408, 'eval_f1': 0.979381443298969, 'eval_runtime': 2.4789, 'eval_samples_per_second': 75.841, 'eval_steps_per_second': 9.682}


In [ ]:
# Visualisations:

!pip install bertviz transformers
!pip install bertviz

In [ ]:
from torch.utils.tensorboard import SummaryWriter
import re
import tensorflow as tf
import tensorboard as tb  #this is important to be able to store embeddings in a format so that I can use tensorflow visualiation

In [ ]:
from transformers import AutoModelForSequenceClassification

'''
This is the code cell where I run into the RAM issue. (and the one below for RoBERTa should be similar)
'''
# The following code saves the hidden states of 50 samples in the test set of all layers of this DistilBERT checkpoint:

# Load model:
fine_tuned_model_distilbert = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/distilbert/checkpoint-165")

model_inputs = tokenizer_distilbert(small_tokenized_dataset_distilbert['test']['text'][0:50], padding=True, truncation=True, return_tensors='pt')
print(len(model_inputs["input_ids"]))
print(len(small_tokenized_dataset_distilbert['test']['text']))
outputs = fine_tuned_model_distilbert(**model_inputs, output_hidden_states=True)


50
188


In [ ]:
import torch
import os
from torch.utils.tensorboard import SummaryWriter
import re
import torch
import tensorflow as tf
import tensorboard as tb

In [ ]:
# Code for visualisation:


path = "results_vis_distilbert" # creates directory, can change it to drive location
layer=0 # sets layer i want to start from
if not os.path.exists(path):
  os.mkdir(path)

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
    sp_token_position = 0
    for token in model_inputs['input_ids'][example]:
      if token != tokenizer_distilbert.pad_token_id and token != tokenizer_distilbert.cls_token_id: # changed this code here a bit
        sp_token_position += 1
      else:
        tensor = outputs['hidden_states'][layer][example][sp_token_position]
        tensors.append(tensor)
        break

    label = [small_tokenized_dataset_distilbert['test']['text'][example],str(small_tokenized_dataset_distilbert['test']['label'][example])]
    labels.append(label)
    example +=1

  writer=SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text','label']) # changed "Text" to "text" and "Emotion" to "label"

  layer+=1

In [ ]:
# Load RoBERTa model:
fine_tuned_model_roberta = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/roberta/checkpoint-165")

model_inputs = tokenizer_roberta(small_tokenized_dataset_roberta['test']['text'][0:50], padding=True, truncation=True, return_tensors='pt')
outputs = fine_tuned_model_roberta(**model_inputs, output_hidden_states=True)

In [ ]:
# The following code saves the hidden states of 50 samples in the test set of all layers of this RoBERTa checkpoint:

path = "results_vis_roberta" # creates directory, can change it to drive location
layer=0 # sets layer i want to start from
if not os.path.exists(path):
  os.mkdir(path)

while layer in range(len(outputs['hidden_states'])):
  if not os.path.exists(path+'/layer_' + str(layer)):
    os.mkdir(path+'/layer_' + str(layer))

  example = 0
  tensors = []
  labels = []

  while example in range(len(outputs['hidden_states'][layer])):
    sp_token_position = 0
    for token in model_inputs['input_ids'][example]:
      if token != tokenizer_roberta.pad_token_id and token != tokenizer_roberta.cls_token_id: # changed this code here a bit
        sp_token_position += 1
      else:
        tensor = outputs['hidden_states'][layer][example][sp_token_position]
        tensors.append(tensor)
        break

    label = [small_tokenized_dataset_roberta['test']['text'][example],str(small_tokenized_dataset_roberta['test']['label'][example])]
    labels.append(label)
    example +=1

  writer=SummaryWriter(path+'/layer_' + str(layer))
  writer.add_embedding(torch.stack(tensors), metadata=labels, metadata_header=['text','label']) # changed "Text" to "text" and "Emotion" to "label"

  layer+=1